# The `input` events

*SPDX-License-Identifier: 0BSD*

The events associated with the `input` builtin are:

- `builtins.input`
- `builtins.input/result`

In [1]:
import subaudit

This works when using a terminal to interact with a the vanilla `python` REPL:

In [2]:
with subaudit.listening('builtins.input', print):
    with subaudit.listening('builtins.input/result', print):
        input('What is your name? ')

It doesn't work in a notebook, because the `input` function is monkey-patched in Jupyter notebooks:

In [3]:
help(input)

Help on method raw_input in module ipykernel.kernelbase:

raw_input(prompt='') method of ipykernel.ipkernel.IPythonKernel instance
    Forward raw_input to frontends
    
    Raises
    ------
    StdinNotImplementedError if active frontend doesn't support stdin.



The situation becomes more interesting when standard input is:

- piped or redirected on the command line, *or*

- "redirected" while the program runs, by patching `sys.stdin`, *or*

- redirected while the program runs, by closing file descriptor 0 and either
  reusing it to open a file on it or duplicating another file descriptor to it
  (`dup2`).

Currently, in CPython, the `input` builtin only raises the
`builtins.input/result` event when GNU readline is used to receive input
interactively. That happens when all the following hold:

- `sys.stdin` wraps actual standard input.
- Standard input is a terminal.
- `sys.stdout` wraps actual standard output.
- Standard output is a terminal.

See https://github.com/python/cpython/blob/v3.11.3/Python/bltinmodule.c#L2249-L2253 and the preceding code.